In [1]:
from mbientlab.metawear import MetaWear, libmetawear, parse_value
import mbientlab.metawear.cbindings as cbindings
from ctypes import Structure, c_void_p, cast, POINTER, c_int, c_float
from threading import Event
from time import sleep
import pandas as pd

In [10]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'

# Create and connect the device
device = MetaWear(device_address)
device.connect()

In [11]:
timestamps = []
x_accs = []
y_accs = []
z_accs = []
x_gyros = []
y_gyros = []
z_gyros = []

def data_handler(context, data):
    values = parse_value(data, n_elem = 2)
    timestamp = int(data.contents.epoch)
    print("timestamp: %.1f, acc: (%.4f,%.4f,%.4f), gyro; (%.4f,%.4f,%.4f)" % (timestamp, values[0].x, values[0].y, values[0].z, values[1].x, values[1].y, values[1].z))
    timestamps.append(timestamp)
    x_accs.append(values[0].x)
    y_accs.append(values[0].y)
    z_accs.append(values[0].z)
    x_gyros.append(values[1].x)
    y_gyros.append(values[1].y)
    z_gyros.append(values[1].z)


In [12]:
e = Event()
callback = cbindings.FnVoid_VoidP_DataP(data_handler)

# Callback for processor creation
def processor_created(context, pointer):
    global processor
    processor = pointer
    e.set()

# Function pointer
fn_wrapper = cbindings.FnVoid_VoidP_VoidP(processor_created)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)

# Fuse the signals together
signals = (c_void_p * 1)()
signals[0] = gyro
libmetawear.mbl_mw_dataprocessor_fuser_create(acc, signals, 1, None, fn_wrapper)
e.wait()

# Subscribe to it
libmetawear.mbl_mw_datasignal_subscribe(processor, None, callback)

# Start the gyro and acc
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)
libmetawear.mbl_mw_acc_start(device.board)

# Stream data for a duration (e.g., 30 seconds)
sleep(10)

# Stop and clean up
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(processor)
libmetawear.mbl_mw_debug_disconnect(device.board)
print("Disconnected and cleaned up")

timestamp: 1714048487911.0, acc: (0.7814,-0.0583,-0.5933), gyro; (-10.3659,4.8780,-0.9146)
timestamp: 1714048487913.0, acc: (0.8010,-0.0657,-0.6063), gyro; (-9.8780,-1.3415,-2.7439)
timestamp: 1714048487913.0, acc: (0.8038,-0.0631,-0.6177), gyro; (-4.2073,-4.8780,0.4878)
timestamp: 1714048487915.0, acc: (0.8326,-0.0681,-0.6224), gyro; (1.1585,0.1220,7.1341)
timestamp: 1714048487916.0, acc: (0.8447,-0.0797,-0.6106), gyro; (1.3415,5.2439,10.6707)
timestamp: 1714048487971.0, acc: (0.8330,-0.1060,-0.6016), gyro; (0.1829,8.7195,8.4146)
timestamp: 1714048487972.0, acc: (0.8082,-0.1523,-0.5909), gyro; (3.1707,13.2317,7.0122)
timestamp: 1714048487973.0, acc: (0.7932,-0.1823,-0.5809), gyro; (4.5122,10.3049,12.3780)
timestamp: 1714048487975.0, acc: (0.7952,-0.0461,-0.5744), gyro; (7.5610,5.0610,39.0854)
timestamp: 1714048487976.0, acc: (0.7834,-0.0714,-0.5904), gyro; (13.9024,5.3049,60.5488)
timestamp: 1714048488031.0, acc: (0.7785,-0.0918,-0.6146), gyro; (3.1098,13.5366,28.2927)
timestamp: 1714

In [13]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

we are disconnected!


In [14]:


#data_array = np.array([timestamps, x_accs, y_accs, z_accs, x_gyros, y_gyros, z_gyros])
data_df = pd.DataFrame(columns=['time', 'x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro'])

data_df['time'] = timestamps
data_df['x_acc'] = x_accs
data_df['y_acc'] = y_accs
data_df['z_acc'] = z_accs
data_df['x_gyro'] = x_gyros
data_df['y_gyro'] = y_gyros
data_df['z_gyro'] = z_gyros

In [15]:
data_df.head(40)

,time,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,1.714048e+12,0.781433,-0.058350,-0.593262,-10.365854,4.878049,-0.914634
1,1.714048e+12,0.801025,-0.065674,-0.606262,-9.878049,-1.341463,-2.743902
2,1.714048e+12,0.803772,-0.063110,-0.617737,-4.207317,-4.878049,0.487805
3,1.714048e+12,0.832581,-0.068115,-0.622375,1.158537,0.121951,7.134147
4,1.714048e+12,0.844666,-0.079651,-0.610596,1.341463,5.243903,10.670732
5,1.714048e+12,0.833008,-0.106018,-0.601624,0.182927,8.719512,8.414635
6,1.714048e+12,0.808228,-0.152283,-0.590881,3.170732,13.231708,7.012195
7,1.714048e+12,0.793213,-0.182251,-0.580872,4.512195,10.304878,12.378049
8,1.714048e+12,0.795227,-0.046082,-0.574402,7.560976,5.060976,39.085365
9,1.714048e+12,0.783447,-0.071411,-0.590393,13.902439,5.304878,60.548782


In [19]:
for i in range(len(timestamps)-1):
    print(timestamps[i+1] - timestamps[i])

2.0
0.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
3.0
0.0
1.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
56.0
1.0
1.0
1.0
55.0
2.0
1.0
1.0
57.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
56.0
3.0
0.0
1.0
54.0
2.0
1.0
1.0
1.0
56.0
1.0
1.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
3.0
0.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
3.0
0.0
1.0
1.0
55.0
2.0
1.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
61.0
2.0
0.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
2.0
1.0
1.0
1.0
55.0
2.0
1.0
1.0
1.0
55.0
2.0
1.0
1.0
57.0
1.0
1.0
2.0
1.0
55.0
3.0
0.0
1.0
1.0
55.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
56.0
1.0
1.0
1.0
2.0
54.0
2.0
1.0
1.0
1.0
56.0
2.0
2.0
2.0
1.0
52.0
1.0
2.0
1.0
57.0
1.0
1.0
2.0
1.0
55.0
3.0
0.0
1.0
1.0
55.0
1.0
58.0
1.0
2.0
1.0
2.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
1.0
2.0
1.0
1.0
55.0
61.0
1.0
1.0
2.0
1.0
55.0
1.0
1.0
2.0
1.0
55.0
1.0
1